In [19]:
import cv2
import numpy as np
t=[[]]
blurValue=7
threshold=10
count=0
x=1000
y=1000
x1=[]
x2=[]
y1=[]
y2=[]
im=[]
l=[]
prediction=[]
import pickle
clf = pickle.load(open('model2.sav', 'rb'))
camera = cv2.VideoCapture(1)
while True:
    ret, image = camera.read()
    cv2.imshow('original', image)
    k = cv2.waitKey(1)
    if k == ord('c'):
        camera.release()
        cv2.destroyAllWindows()
        #image = cv2.resize(image, (28,28), interpolation = cv2.INTER_AREA)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
        #ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY_INV)
        #kernel = np.ones((3,3),np.uint8)
        thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, blockSize=25, C=11)
        #opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        #thresh = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            l.append(cv2.contourArea(cnt))
        l.sort(reverse=True)
        for cnt in contours:
            if cv2.contourArea(cnt)<(l[0]/80):
                q,y,w,h = cv2.boundingRect(cnt)
                for i in range(y,y+h):
                    for j in range(q,q+w):
                        thresh[i][j]=0
        for i in range(thresh.shape[1]):
            old=count
            for j in range(thresh.shape[0]):
                if thresh[j][i]==255:
                    count+=1
            if count>old:
                x=min(x,i)
            if count>0 and old==count:
                x2.append(i)
                x1.append(x)
                x=1000
                count=0
        if old!=count:
            x2.append(thresh.shape[1])
            x1.append(x)
        count=0
        for z in range(len(x1)):
            for i in range(thresh.shape[0]):
                old=count
                for j in range(x1[z],x2[z]):
                    if thresh[i][j]==255:
                        count+=1
                if count>old:
                    y=min(y,i)
                if count>0 and old==count:
                    y2.append(i)
                    y1.append(y)
                    y=1000
                    count=0
            if old!=count:
                y2.append(thresh.shape[0])
                y1.append(y)
        for i in range(len(x1)):
            imcrop = thresh[y1[i]:y2[i], x1[i]:x2[i]]
            a=45-y2[i]+y1[i]
            b=45-x2[i]+x1[i]
            a=max(max(a,0),(x2[i]-x1[i])-(y2[i]-y1[i]))
            b=max(max(b,0),(y2[i]-y1[i])-(x2[i]-x1[i]))
            constan = cv2.copyMakeBorder(imcrop,int(a/2),a-int(a/2),int(b/2),b-int(b/2),cv2.BORDER_CONSTANT)
            imag = cv2.resize(constan, (45,45), interpolation = cv2.INTER_AREA)
            ret, thr = cv2.threshold(imag, threshold, 255, cv2.THRESH_BINARY)
            #thr = cv2.adaptiveThreshold(imag, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize=5, C=5)
            kernel = np.ones((2,2), np.uint8)
            thr = cv2.dilate(thr, kernel, iterations=1)
            for l in range(45):
                for j in range(45):
                    t[0].append(int(thr[l][j]/255))
            predicted = clf.predict(t)
            prediction.append(predicted[0])
            print(predicted)
            t=[[]]
        break
    elif k == 27:  # press ESC to exit
        break
cv2.destroyAllWindows()

/home/youngdumb/ml/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/youngdumb/ml/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[13]
[13]
[1]


In [ ]:
j=[]
d=[]
i=0
s=''
while i < len(prediction):
    c=0
    while i<len(prediction) and prediction[i]<=9:
        c=c*10+prediction[i]
        i+=1
    d.append(c)
    s+=str(c)
    if i<len(prediction):
        j.append(prediction[i])
        if prediction[i]==10:
            s+=' / '
        if prediction[i]==11:
            s+=' * '
        if prediction[i]==12:
            s+=' + '
        if prediction[i]==13:
            s+=' - '
    i+=1

ans = d[0]
for i in range(0,len(j)):
    if j[i]==10:
        ans = ans/d[i+1]
    if j[i]==11:
        ans = ans*d[i+1]
    if j[i]==12:
        ans = ans+d[i+1]
    if j[i]==13:
        ans = ans-d[i+1]
print(ans)
s+=' = '+str(ans)    
    
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(image,s,(30,450), font, 2,(255,255,255),2,cv2.LINE_AA)

cv2.imshow('original', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
cv2.imshow('original', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
print(x1,x2,y1,y2)

[38, 210, 462] [149, 395, 584] [58, 155, 191, 227, 104] [303, 177, 209, 249, 257]
